In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import json
import pandas as pd
import numpy as np
import pickle

In [2]:
with open("./dataset/dataset.json", "rb") as file :
    data = json.load(file)

In [3]:
tags = []
inputs = []
responses={}
classes = []

for intent in data['intents']:
  if intent["tag"] not in classes :
    classes.append(intent["tag"])
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [7]:
pickle.dump(classes, open('classes.pkl', 'wb'))
pickle.dump(responses, open('responses.pkl', 'wb'))

In [ ]:
import string

for i in range(0, len(inputs)) :
    inputs[i] = inputs[i].translate(str.maketrans('', '', string.punctuation)).lower()

In [ ]:
df = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
print(df.head())

In [ ]:
df["tags"].value_counts()

In [ ]:
df.tags = df.tags.replace({"greeting": 0, "goodbye": 1, "thanks": 2, "creator": 3, "name": 4, "skill": 5, "funny": 6, "disease": 7})

In [ ]:
df.head()

In [ ]:
df["tags"].value_counts()

In [ ]:
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]

In [ ]:
X

In [ ]:
y

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(len(classes), activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.sparse_categorical_crossentropy

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=["acc"])

In [ ]:
model.fit(X, y, epochs=150)

In [ ]:
sentence = "Hello!"

sentence = sentence.translate(str.maketrans('', '', string.punctuation))

sent_seq = np.expand_dims(sentence, axis = 0)

pred = model.predict(sent_seq)

pred_class = np.argmax(pred)

print(classes[pred_class])

In [ ]:
model.save("model_saved.keras")